Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [68]:
batch_size = 128
patch_size = 3
depth1 = 8
depth2 = 16
depth3 = 32
num_hidden1 = 256
num_hidden2 = 128

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth1, depth2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth2, depth3], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[depth3]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [4 * 4 * depth3, num_hidden1], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden1, num_hidden2], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))
  layer6_weights = tf.Variable(tf.truncated_normal(
      [num_hidden2, num_labels], stddev=0.1))
  layer6_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  keep_prob = tf.placeholder(tf.float32)
  
  # Model.
  def model(data, dropout=True):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    if dropout:
      hidden = tf.nn.dropout(hidden, keep_prob)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    if dropout:
      hidden = tf.nn.dropout(hidden, keep_prob)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layer3_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer3_biases)
    if dropout:
      hidden = tf.nn.dropout(hidden, keep_prob)
    pool = tf.nn.avg_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
    if dropout:
      hidden = tf.nn.dropout(hidden, keep_prob)
    
    hidden = tf.nn.relu(tf.matmul(hidden, layer5_weights) + layer5_biases)
    if dropout:
      hidden = tf.nn.dropout(hidden, keep_prob)
    
    return tf.matmul(hidden, layer6_weights) + layer6_biases
  
  # Training computation.
  logits = model(tf_train_dataset, False)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  # optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, False))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, False))

In [71]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.162816
Minibatch accuracy: 9.4%
Validation accuracy: 10.2%
Minibatch loss at step 100: 1.207764
Minibatch accuracy: 61.7%
Validation accuracy: 63.2%
Minibatch loss at step 200: 0.714417
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 300: 0.660759
Minibatch accuracy: 77.3%
Validation accuracy: 81.8%
Minibatch loss at step 400: 0.622693
Minibatch accuracy: 80.5%
Validation accuracy: 82.9%
Minibatch loss at step 500: 0.404937
Minibatch accuracy: 86.7%
Validation accuracy: 84.5%
Minibatch loss at step 600: 0.583618
Minibatch accuracy: 82.8%
Validation accuracy: 84.8%
Minibatch loss at step 700: 0.556175
Minibatch accuracy: 83.6%
Validation accuracy: 85.5%
Minibatch loss at step 800: 0.261043
Minibatch accuracy: 92.2%
Validation accuracy: 85.8%
Minibatch loss at step 900: 0.432247
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 1000: 0.586296
Minibatch accuracy: 83.6%
Validation accuracy: 86.6%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

Unmodified network test accuracy: 90.0% (1,001 steps) and 94.5% (10,001 steps)

Using max pooling test accuracy: 91.0% (1,001 steps) and 95.0% (10,001 steps)

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

Add Dropout to the dense layer, the test accuracy is 89.1% (1001 steps).

Increase batch size from 16 to 100, depth from 16 to 32, hidden layer size from 64 to 512. Test accuracy: 92.5%

Add second depth parameter (64). Test accuracy: 92.3%

Increase hidden nodes to 1024 and steps to 20,001. Test accuracy: 96.2%

Lower batch size to 50 and use AdamOptimizer. Test accuracy: 96.0%

Revert to original max_pool model. Test accuracy: 90.0% (1,0001 steps)

batch_size:50, patch_size:5, depth1:16, depth2:16, num_hidden:1024, dropout:True, 20001 steps. Test accuracy: 95.5%

batch_size:50, patch_size:5, depth1:16, depth2:64, num_hidden:64, dropout:True, 20001 steps. Test accuracy: 96.0%

Add Dropout to two convolutional layers, reset other settings. 1001 steps. Test accuracy: 91.0%

batch_size:50, patch_size:5, depth1:32, depth2:64, num_hidden:1024, dropout:True, AdamOptimizer, 1001 steps.

----------------

Add 3rd convolutional layer and 2nd densely connected layer to more closely replicate LeNet5.

Adjust all parameters. 10,001 steps. Test accuracy: 95.3%

Change 3rd convolutional layer to use avg_pool. Switch to AdamOptimizer with learning=0.001. Test accuracy: 96.5% **<-- Highest Score**